# Load in data 
+ Data comes as a tar file
+ I opted to just load in files into pandas instead of attempting to unwrap that tar file in python
+ We have a file containing all of the edges facebook.csv
+ What we need to do is combine the .feat files with the .featnames files
+ below you can see what each of these files looks like

In [1]:
import pandas as pd 
import numpy as np

In [2]:
example= pd.read_csv('0.feat',sep=" ",header=None)
print(example.shape)
example.head()


(347, 225)


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,224
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
example= pd.read_csv('0.featnames',sep=" ",header=None)
print(example.shape)
example.head()

(224, 4)


,0,1,2,3
0,0,birthday;anonymized,feature,0
1,1,birthday;anonymized,feature,1
2,2,birthday;anonymized,feature,2
3,3,birthday;anonymized,feature,3
4,4,birthday;anonymized,feature,4


# structure of our data

+  Column values in the .feat files(225 columns) are the equivalent index value for our .featnames files(224 rows)
    + The difference of 1 in the shape is due to the 1st column in the .feat file representing our node value and not a feature value
+ First goal is to join the names from .featnames in the feat files


### Build names lists
+ our files are stored as a random int.name
    + I.E  '0.featnames' and '0.feat'
     + Below we use list comprehension to build out file names


In [4]:
names=["0","107","1684","1912","3437",'348',"3980","414","686","698"]
feat_file_names= [x+".feat" for x in names]
featnames_filenames=[x+".featnames" for x in names]

In [5]:
# check names
featnames_filenames

['0.featnames',
 '107.featnames',
 '1684.featnames',
 '1912.featnames',
 '3437.featnames',
 '348.featnames',
 '3980.featnames',
 '414.featnames',
 '686.featnames',
 '698.featnames']

### Load files into pandas 

In [6]:
list_feat_names=[]
list_features=[]
for i,x in enumerate(featnames_filenames):
    list_feat_names.append(pd.read_csv(x,sep=" ",header=None))
    list_features.append(pd.read_csv(feat_file_names[i],sep=" ",header=None))

### Check shape matches earlier shape 

In [7]:
list_feat_names[0].shape

(224, 4)

In [8]:
list_features[0].shape

(347, 225)

### Practice on one set of each document

In [9]:
df=list_features[0]
df.head()
indexes=df.iloc[:,0]
df=df.iloc[:,1:]
df.columns= list_feat_names[0][1]
df["node"]=indexes
df.head()


1,birthday;anonymized,birthday;anonymized,birthday;anonymized,birthday;anonymized,birthday;anonymized,birthday;anonymized,birthday;anonymized,birthday;anonymized,education;classes;id;anonymized,education;classes;id;anonymized,...,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;start_date;anonymized,work;with;id;anonymized,node
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


# Build df's as list

In [10]:
final_product=[]
for i,x in enumerate(list_features):
    df=x
    indexes=x.iloc[:,0]
    df=x.iloc[:,1:]
    df.columns= list_feat_names[i][1].astype(str)+'_'+list_feat_names[i][3].astype(str)
  #  df["nodes"]=indexes
    df.index=indexes
    final_product.append(df)
    
    

## check shape of list of df

In [11]:
for x in final_product:
    print(x.shape)

(347, 224)
(1045, 576)
(792, 319)
(755, 480)
(547, 262)
(227, 161)
(59, 42)
(159, 105)
(170, 63)
(66, 48)


## Build dictionary from df

In [12]:
my_nodes={}
for node in range(1,4036):
    my_nodes.setdefault(node, [])


for x in final_product:
    n=0
    for node in range(1,4036):
        for index in x.index:
            if index==node:
                my_nodes[node].append(x.loc[node,])
                

#my_nodes[node].append({x.loc[node,][0].index: x.loc[node,][0].values})

## Check for duplicate values

In [13]:
index_list=[]
duplicates_index=[]
total_rows=0
for x in final_product:
    for index in x.index:
        total_rows+=1
        if index not in index_list:
            index_list.append(index)
        else:
            duplicates_index.append(index)


len(duplicates_index)

132

In [14]:
import os
os.getcwd()

'E:\\facebook'

In [17]:
var = []
var1 = []
for i in range(len(featnames_filenames)):
    c = pd.read_csv(featnames_filenames[i], header=None,)[0].tolist()
    for j in range(len(c)):
        var1.append(' '.join(c[j].split(' ')[1:-2]))
    for v in range(len(var1)):
        var.append(' '.join(var1[v].split(';')[:-1]))
var = pd.Series(var).unique().tolist()
var

['birthday',
 'education classes id',
 'education concentration id',
 'education degree id',
 'education school id',
 'education type',
 'education with id',
 'education year id',
 'first_name',
 'gender',
 'hometown id',
 'languages id',
 'last_name',
 'locale',
 'location id',
 'work employer id',
 'work end_date',
 'work location id',
 'work position id',
 'work start_date',
 'work with id',
 'middle_name',
 'work from id',
 'work projects id',
 'religion',
 'name',
 'political']

# Build dictionary of Attribute category: categorical answer
+ Apparently some nodes are not present in dataset, so i had to run try except 
+ I know its not exactly in the form we need but im sure its step or two away

In [31]:
blank_nodes={}
n=0
for node in range(1,4036):
    blank_nodes.setdefault(node, [])


for node in range(1,4036):
    print(n)
    try:
        node_col_names = my_nodes[node][0].index
        node_col_vals = my_nodes[node][0].values
        for i,x in enumerate(node_col_vals):
            if x==1:
                blank_nodes[node].append(node_col_names[i].split("anonymized_"))
    except:
        continue
    n+=1
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204


3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946


In [34]:
blank_nodes



{1: [['gender;', '77'], ['locale;', '127']],
 2: [['education;school;id;', '35'],
  ['education;type;', '53'],
  ['education;type;', '55'],
  ['education;year;id;', '57'],
  ['gender;', '78'],
  ['languages;id;', '92'],
  ['languages;id;', '98'],
  ['last_name;', '114'],
  ['locale;', '126'],
  ['location;id;', '135']],
 3: [['birthday;', '7'],
  ['education;concentration;id;', '14'],
  ['education;school;id;', '34'],
  ['education;school;id;', '50'],
  ['education;type;', '53'],
  ['education;type;', '55'],
  ['education;year;id;', '59'],
  ['education;year;id;', '65'],
  ['gender;', '78'],
  ['languages;id;', '92'],
  ['locale;', '127'],
  ['location;id;', '137'],
  ['work;end_date;', '168'],
  ['work;end_date;', '170'],
  ['work;location;id;', '137'],
  ['work;start_date;', '164'],
  ['work;start_date;', '202']],
 4: [['education;school;id;', '50'],
  ['education;type;', '53'],
  ['education;type;', '55'],
  ['education;with;id;', '56'],
  ['gender;', '78'],
  ['locale;', '127']],
 

# Build blank df of NAN

## Prove col names are same

In [15]:
# my_list=[x for x in my_nodes[1][0].index]
# my_list2= [x for x in df.iloc[0,:].index]

# for x in my_list:
#     if x in my_list2:
#         print(x)

birthday;anonymized_2
birthday;anonymized_3
education;concentration;id;anonymized_14
education;degree;id;anonymized_22
education;type;anonymized_53
education;type;anonymized_54
education;type;anonymized_55
education;year;id;anonymized_59
education;year;id;anonymized_60
education;year;id;anonymized_61
education;year;id;anonymized_64
education;year;id;anonymized_71
education;year;id;anonymized_72
gender;anonymized_77
gender;anonymized_78
languages;id;anonymized_92
locale;anonymized_126
locale;anonymized_127
work;employer;id;anonymized_140
work;employer;id;anonymized_154
work;end_date;anonymized_157
work;end_date;anonymized_161
work;start_date;anonymized_157
work;start_date;anonymized_196


## Build blank df 


## trying to build df's w concat

In [16]:
#df = pd.DataFrame(np.nan, index=np.arange(1,4040), columns=col_list)

# new_df=pd.DataFrame(data = [my_nodes[1][0].values] * len(my_nodes[1][0]), columns = my_nodes[1][0].index)
# df.merge(new_df, left_index=True, right_index=True)
         

# df.merge(my_nodes[1][0].to_frame().T, left_index=True, right_index=True)
# my_nodes[1][0].to_frame().T
# new_df= pd.concat([final_product[0],final_product[1],final_product[2],final_product[3],final_product[4],final_product[5],final_product[6],final_product[7],final_product[8],final_product[9]])
# df_9=pd.concat([final_product[0],final_product[1],final_product[2]])